In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Loading the Data

In [2]:
flights = pd.read_csv('nycflights.csv')

In [3]:
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,1/1/2013 5:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,1/1/2013 5:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,1/1/2013 5:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,1/1/2013 5:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,1/1/2013 6:00


In [4]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 19 columns):
year              336776 non-null int64
month             336776 non-null int64
day               336776 non-null int64
dep_time          328521 non-null float64
sched_dep_time    336776 non-null int64
dep_delay         328521 non-null float64
arr_time          328063 non-null float64
sched_arr_time    336776 non-null int64
arr_delay         327346 non-null float64
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null float64
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
dtypes: float64(5), int64(9), object(5)
memory usage: 48.8+ MB


We load the data, take a look at the head, and the descriptions.

It seems that dep_time, dep_delay, arr_time, arr_delay, and air_time were all converted into floats. These are supposed to represent times where '1455' represent the time '14:55'. There should not be any decimal values. The fact that decimals are appearing means that something happened in the import that made python believe they were floating point numbers.

In [5]:
flights.tail()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,3393,NaN,JFK,DCA,NaN,213,14,55,9/30/2013 14:00
336772,2013,9,30,NaN,2200,NaN,NaN,2312,NaN,9E,3525,NaN,LGA,SYR,NaN,198,22,0,9/30/2013 22:00
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,3461,N535MQ,LGA,BNA,NaN,764,12,10,9/30/2013 12:00
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,3572,N511MQ,LGA,CLE,NaN,419,11,59,9/30/2013 11:00
336775,2013,9,30,NaN,840,NaN,NaN,1020,NaN,MQ,3531,N839MQ,LGA,RDU,NaN,431,8,40,9/30/2013 8:00


An inspection of the tail shows that there are missing values in dep_time and the other columns. Because python represents missing numeric values with NaN, they were convered to floating point numbers.

We will reload the flights csv file, this time specifying the offending columns as 'object' type. (It's a little weird, but we do not use str type because str uses NumPy's str type which is different from regular Python strings. To use Python strings, Pandas uses 'object' type)

## Loading the data again and doing a little bit of clean up

In [6]:
flights = pd.read_csv('nycflights.csv', dtype = {'dep_time' : object, 
                                                 'dep_delay': object, 
                                                 'arr_time' : object, 
                                                 'arr_delay': object, 
                                                 'air_time' : object})

In [7]:
flights.head() # the dep_time column no longer has any decimals in it.

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,1/1/2013 5:00
1,2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,1/1/2013 5:00
2,2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,1/1/2013 5:00
3,2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,1/1/2013 5:00
4,2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,1/1/2013 6:00


In [8]:
flights.tail(10) # the missing values are still preserved

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
336766,2013,9,30,2240,2250,-10,2347,7,-20,B6,2002,N281JB,JFK,BUF,52,301,22,50,9/30/2013 22:00
336767,2013,9,30,2241,2246,-5,2345,1,-16,B6,486,N346JB,JFK,ROC,47,264,22,46,9/30/2013 22:00
336768,2013,9,30,2307,2255,12,2359,2358,1,B6,718,N565JB,JFK,BOS,33,187,22,55,9/30/2013 22:00
336769,2013,9,30,2349,2359,-10,325,350,-25,B6,745,N516JB,JFK,PSE,196,1617,23,59,9/30/2013 23:00
336770,2013,9,30,NaN,1842,NaN,NaN,2019,NaN,EV,5274,N740EV,LGA,BNA,NaN,764,18,42,9/30/2013 18:00
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,3393,NaN,JFK,DCA,NaN,213,14,55,9/30/2013 14:00
336772,2013,9,30,NaN,2200,NaN,NaN,2312,NaN,9E,3525,NaN,LGA,SYR,NaN,198,22,0,9/30/2013 22:00
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,3461,N535MQ,LGA,BNA,NaN,764,12,10,9/30/2013 12:00
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,3572,N511MQ,LGA,CLE,NaN,419,11,59,9/30/2013 11:00
336775,2013,9,30,NaN,840,NaN,NaN,1020,NaN,MQ,3531,N839MQ,LGA,RDU,NaN,431,8,40,9/30/2013 8:00


The next thing I notice is that the dates are split across three columns: year, month, day.

I will use Python's date functions to combine the three values into a single date value

In [9]:
flights['date'] = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day})

In [10]:
flights.head() # we have now appeneded a new column that shows the date

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,date
0,2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,1/1/2013 5:00,2013-01-01
1,2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,1/1/2013 5:00,2013-01-01
2,2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,1/1/2013 5:00,2013-01-01
3,2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,1/1/2013 5:00,2013-01-01
4,2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,1/1/2013 6:00,2013-01-01


In [11]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 20 columns):
year              336776 non-null int64
month             336776 non-null int64
day               336776 non-null int64
dep_time          328521 non-null object
sched_dep_time    336776 non-null int64
dep_delay         328521 non-null object
arr_time          328063 non-null object
sched_arr_time    336776 non-null int64
arr_delay         327346 non-null object
carrier           336776 non-null object
flight            336776 non-null int64
tailnum           334264 non-null object
origin            336776 non-null object
dest              336776 non-null object
air_time          327346 non-null object
distance          336776 non-null int64
hour              336776 non-null int64
minute            336776 non-null int64
time_hour         336776 non-null object
date              336776 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(9), object(10)
memory usage: 51.4+ MB

With the times expressed as they are, we can't calculate the real time difference between columns. The time between '600' and '545' is really only 15 minutes, but the math will make the wrong calculation of 55. Thus, we need to split our text into hour and minute and make them into date-time values.

In [12]:
# we filter the dep_time to just non_null values
# I will extract the minute values by treating the series as a string and pulling the last two characters
# we get the hour by taking everything but the last two characters
dep_min  = flights.dep_time[ flights.dep_time.notnull() ].astype(str).str[-2:  ]
dep_hour = flights.dep_time[ flights.dep_time.notnull() ].astype(str).str[  :-2]
print(dep_hour.head())
print(dep_min.head())

0    5
1    5
2    5
3    5
4    5
Name: dep_time, dtype: object
0    17
1    33
2    42
3    44
4    54
Name: dep_time, dtype: object


In [13]:
dep_date_time = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day, 
                                'hour': dep_hour, 'minute': dep_min})

In [14]:
print(dep_date_time.head(10))
print(dep_date_time.tail())

0   2013-01-01 05:17:00
1   2013-01-01 05:33:00
2   2013-01-01 05:42:00
3   2013-01-01 05:44:00
4   2013-01-01 05:54:00
5   2013-01-01 05:54:00
6   2013-01-01 05:55:00
7   2013-01-01 05:57:00
8   2013-01-01 05:57:00
9   2013-01-01 05:58:00
dtype: datetime64[ns]
336771   NaT
336772   NaT
336773   NaT
336774   NaT
336775   NaT
dtype: datetime64[ns]


I'd like to do the same kind of conversion for all of the other time columns. 

Instead of copying and pasting the code and modifying it for each column, I'm going to build a function.

In [15]:
def convert_time(column_name):
    the_min = flights[column_name][ flights[column_name].notnull() ].astype(str).str[-2:  ]
    the_hr  = flights[column_name][ flights[column_name].notnull() ].astype(str).str[  :-2]
    date_time = pd.to_datetime({'year':flights.year, 'month':flights.month, 'day':flights.day, 
                                'hour': the_hr, 'minute': the_min})
    # the magic of indexes allow us to combine flights.year which has all the rows
    # and the_hr which only exists for rows where dep_time is not NaN
    return date_time

In [16]:
# let's test out the function
dep_date_time = convert_time('dep_time')

In [17]:
print(dep_date_time.head(10))
print(dep_date_time.tail(10))

0   2013-01-01 05:17:00
1   2013-01-01 05:33:00
2   2013-01-01 05:42:00
3   2013-01-01 05:44:00
4   2013-01-01 05:54:00
5   2013-01-01 05:54:00
6   2013-01-01 05:55:00
7   2013-01-01 05:57:00
8   2013-01-01 05:57:00
9   2013-01-01 05:58:00
dtype: datetime64[ns]
336766   2013-09-30 22:40:00
336767   2013-09-30 22:41:00
336768   2013-09-30 23:07:00
336769   2013-09-30 23:49:00
336770                   NaT
336771                   NaT
336772                   NaT
336773                   NaT
336774                   NaT
336775                   NaT
dtype: datetime64[ns]


In [18]:
# seems to work!
# let's go ahead and use it:
# dep_date_time       = convert_time('dep_time')
sched_dep_date_time = convert_time('sched_dep_time')
arr_date_time       = convert_time('arr_time')
sched_arr_date_time = convert_time('sched_arr_time')

In [19]:
# pd.concat creates a new dataframe by joining several pandas objects together
datetimes = pd.concat({'dep_date_time': dep_date_time,
           'sched_dep_date_time': sched_dep_date_time, 
           'arr_date_time':arr_date_time, 
           'sched_arr_date_time':sched_arr_date_time}, axis = 1)
datetimes.head()

,dep_date_time,sched_dep_date_time,arr_date_time,sched_arr_date_time
0,2013-01-01 05:17:00,2013-01-01 05:15:00,2013-01-01 08:30:00,2013-01-01 08:19:00
1,2013-01-01 05:33:00,2013-01-01 05:29:00,2013-01-01 08:50:00,2013-01-01 08:30:00
2,2013-01-01 05:42:00,2013-01-01 05:40:00,2013-01-01 09:23:00,2013-01-01 08:50:00
3,2013-01-01 05:44:00,2013-01-01 05:45:00,2013-01-01 10:04:00,2013-01-01 10:22:00
4,2013-01-01 05:54:00,2013-01-01 06:00:00,2013-01-01 08:12:00,2013-01-01 08:37:00


In [20]:
flights = flights.join(datetimes)  # we use the .join() function

In [21]:
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,air_time,distance,hour,minute,time_hour,date,dep_date_time,sched_dep_date_time,arr_date_time,sched_arr_date_time
0,2013,1,1,517,515,2,830,819,11,UA,...,227,1400,5,15,1/1/2013 5:00,2013-01-01,2013-01-01 05:17:00,2013-01-01 05:15:00,2013-01-01 08:30:00,2013-01-01 08:19:00
1,2013,1,1,533,529,4,850,830,20,UA,...,227,1416,5,29,1/1/2013 5:00,2013-01-01,2013-01-01 05:33:00,2013-01-01 05:29:00,2013-01-01 08:50:00,2013-01-01 08:30:00
2,2013,1,1,542,540,2,923,850,33,AA,...,160,1089,5,40,1/1/2013 5:00,2013-01-01,2013-01-01 05:42:00,2013-01-01 05:40:00,2013-01-01 09:23:00,2013-01-01 08:50:00
3,2013,1,1,544,545,-1,1004,1022,-18,B6,...,183,1576,5,45,1/1/2013 5:00,2013-01-01,2013-01-01 05:44:00,2013-01-01 05:45:00,2013-01-01 10:04:00,2013-01-01 10:22:00
4,2013,1,1,554,600,-6,812,837,-25,DL,...,116,762,6,0,1/1/2013 6:00,2013-01-01,2013-01-01 05:54:00,2013-01-01 06:00:00,2013-01-01 08:12:00,2013-01-01 08:37:00


Let's go ahead and drop the year, month, and day columns as we don't need them anymore.

In [22]:
flights = flights.drop(['year','month','day'], axis = 1)

In [23]:
flights.head()

,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,...,air_time,distance,hour,minute,time_hour,date,dep_date_time,sched_dep_date_time,arr_date_time,sched_arr_date_time
0,517,515,2,830,819,11,UA,1545,N14228,EWR,...,227,1400,5,15,1/1/2013 5:00,2013-01-01,2013-01-01 05:17:00,2013-01-01 05:15:00,2013-01-01 08:30:00,2013-01-01 08:19:00
1,533,529,4,850,830,20,UA,1714,N24211,LGA,...,227,1416,5,29,1/1/2013 5:00,2013-01-01,2013-01-01 05:33:00,2013-01-01 05:29:00,2013-01-01 08:50:00,2013-01-01 08:30:00
2,542,540,2,923,850,33,AA,1141,N619AA,JFK,...,160,1089,5,40,1/1/2013 5:00,2013-01-01,2013-01-01 05:42:00,2013-01-01 05:40:00,2013-01-01 09:23:00,2013-01-01 08:50:00
3,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,...,183,1576,5,45,1/1/2013 5:00,2013-01-01,2013-01-01 05:44:00,2013-01-01 05:45:00,2013-01-01 10:04:00,2013-01-01 10:22:00
4,554,600,-6,812,837,-25,DL,461,N668DN,LGA,...,116,762,6,0,1/1/2013 6:00,2013-01-01,2013-01-01 05:54:00,2013-01-01 06:00:00,2013-01-01 08:12:00,2013-01-01 08:37:00


If we need to get the month or day, we can extract them using the .dt. object attributes

In [24]:
flights.date.dt.month.head()

0    1
1    1
2    1
3    1
4    1
Name: date, dtype: int64

In [25]:
flights.date.dt.day.tail()

336771    30
336772    30
336773    30
336774    30
336775    30
Name: date, dtype: int64

Now that we've got the data in and cleaned up, let's do some Basic Exploratory Analysis

## Let's start with some basics:

### How many unique carriers are there? How many flights for each carrier?

In [26]:
flights.carrier.unique()

array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

In [27]:
flights.carrier.nunique()

16

In [28]:
carrier_counts = flights.carrier.value_counts()
carrier_counts

UA    58665
B6    54635
EV    54173
DL    48110
AA    32729
MQ    26397
US    20536
9E    18460
WN    12275
VX     5162
FL     3260
AS      714
F9      685
YV      601
HA      342
OO       32
Name: carrier, dtype: int64

In [30]:
carrier_counts.index

Index(['UA', 'B6', 'EV', 'DL', 'AA', 'MQ', 'US', '9E', 'WN', 'VX', 'FL', 'AS',
       'F9', 'YV', 'HA', 'OO'],
      dtype='object')

In [31]:
carrier_counts.values

array([58665, 54635, 54173, 48110, 32729, 26397, 20536, 18460, 12275,
        5162,  3260,   714,   685,   601,   342,    32], dtype=int64)

In [33]:
airlines = pd.read_csv('airlines.csv', index_col = 'carrier')
airlines

,name
carrier,
9E,Endeavor Air Inc.
AA,American Airlines Inc.
AS,Alaska Airlines Inc.
B6,JetBlue Airways
DL,Delta Air Lines Inc.
EV,ExpressJet Airlines Inc.
F9,Frontier Airlines Inc.
FL,AirTran Airways Corporation
HA,Hawaiian Airlines Inc.


In [34]:
airlines_count = airlines.join(carrier_counts)
airlines_count

,name,carrier
carrier,,
9E,Endeavor Air Inc.,18460
AA,American Airlines Inc.,32729
AS,Alaska Airlines Inc.,714
B6,JetBlue Airways,54635
DL,Delta Air Lines Inc.,48110
EV,ExpressJet Airlines Inc.,54173
F9,Frontier Airlines Inc.,685
FL,AirTran Airways Corporation,3260
HA,Hawaiian Airlines Inc.,342


In [35]:
airlines_count.rename(index = str, columns = {"name": "name", "carrier": "count" })

,name,count
carrier,,
9E,Endeavor Air Inc.,18460
AA,American Airlines Inc.,32729
AS,Alaska Airlines Inc.,714
B6,JetBlue Airways,54635
DL,Delta Air Lines Inc.,48110
EV,ExpressJet Airlines Inc.,54173
F9,Frontier Airlines Inc.,685
FL,AirTran Airways Corporation,3260
HA,Hawaiian Airlines Inc.,342


### What about origination airports?

In [ ]:
flights.origin.unique()

In [ ]:
flights.origin.value_counts()  # it appears that the three airports in the new york area are used with similar frequency

In [ ]:
# alternative method using .groupby(), which is similar to dplyr::groupby
flights.groupby('origin').sched_dep_date_time.count()

In [ ]:
# if we perform a count on the dep_date_time column, it does not include NaNs.
flights.groupby('origin').dep_date_time.count()

### Destination airports?

In [ ]:
flights.dest.unique()

In [ ]:
flights.dest.nunique()  # 105 destinations

In [ ]:
flights.dest.value_counts().head(10)  # a quick peek at the top 10 destinations

### Which month is most busy?

In [ ]:
flights.dep_date_time.dt.month.value_counts()

In [ ]:
flights.dep_date_time.dt.month.value_counts().sort_index().plot(kind = 'bar', color = 'dodgerblue')
plt.title('Number of Flights Each Month')
plt.show()

### What time of day is most busy?

In [ ]:
# using groupby instead of value_counts
flights.groupby(dep_date_time.dt.hour).date.count()

In [ ]:
flights.groupby(dep_date_time.dt.hour).date.count().plot(kind = 'bar', color = 'dodgerblue')
plt.title('Flights Each Hour')
plt.show()